In [24]:
%pip install duckdb --upgrade

Note: you may need to restart the kernel to use updated packages.


In [25]:
import duckdb
cursor = duckdb.connect()
print(cursor.execute('SELECT 42').fetchall())

[(42,)]


In [2]:
import duckdb

In [26]:
import pandas as pd

In [3]:
# connect to an in-memory temporary database
conn = duckdb.connect()

In [4]:
# run arbitrary SQL commands
conn.execute("CREATE TABLE test_table (i INTEGER, j STRING)")

In [8]:
# add some data
conn.execute("INSERT INTO test_table VALUES (1, 'one')")

In [11]:
# we can use placeholders for parameters
conn.executemany("INSERT INTO test_table VALUES (?, ?)", [[2, 'two'], [3, 'three']])


In [15]:
# add some data
conn.execute("INSERT INTO test_table VALUES (4, 'four')")

In [16]:
# fetch as pandas data frame
print(conn.execute("SELECT * FROM test_table").fetch_df())

   i      j
0  1    one
1  2    two
2  3  three
3  4   four


In [17]:
# fetch as list of masked numpy arrays, cleaner when handling NULLs
print(conn.execute("SELECT * FROM test_table").fetchnumpy())

{'i': array([1, 2, 3, 4]), 'j': array(['one', 'two', 'three', 'four'], dtype=object)}


In [27]:
# Connect to an in-memory DuckDB instance
con = duckdb.connect(database=':memory:')

In [28]:
# Path to the CSV file
csv_file_path = "C:/Users/Sridevi/Downloads/supermarket_sales.csv"

In [29]:
# The read_csv_auto function infers the schema automatically
query = f"""
SELECT *
FROM read_csv_auto('{csv_file_path}')
"""


In [31]:
# Execute the query and fetch the results into a Pandas DataFrame
df = con.execute(query).fetchdf()

In [32]:
print(df)

      Invoice ID Branch       City Customer type  Gender  \
0    750-67-8428      A     Yangon        Member  Female   
1    226-31-3081      C  Naypyitaw        Normal  Female   
2    631-41-3108      A     Yangon        Normal    Male   
3    123-19-1176      A     Yangon        Member    Male   
4    373-73-7910      A     Yangon        Normal    Male   
..           ...    ...        ...           ...     ...   
995  233-67-5758      C  Naypyitaw        Normal    Male   
996  303-96-2227      B   Mandalay        Normal  Female   
997  727-02-1313      A     Yangon        Member    Male   
998  347-56-2442      A     Yangon        Normal    Male   
999  849-09-3807      A     Yangon        Member  Female   

               Product line  Unit price  Quantity   Tax 5%      Total  \
0         Health and beauty       74.69         7  26.1415   548.9715   
1    Electronic accessories       15.28         5   3.8200    80.2200   
2        Home and lifestyle       46.33         7  16.2155  

In [33]:
query_1 = f"""
SELECT *
FROM read_csv_auto('{csv_file_path}')
ORDER BY Total DESC
LIMIT 1
"""

In [34]:
df_1 = con.execute(query_1).fetchdf()

In [35]:
print(df_1)

    Invoice ID Branch       City Customer type  Gender         Product line  \
0  860-79-0874      C  Naypyitaw        Member  Female  Fashion accessories   

   Unit price  Quantity  Tax 5%    Total       Date   Time      Payment  \
0        99.3        10   49.65  1042.65 2019-02-15  14:53  Credit card   

    cogs  gross margin percentage  gross income  Rating  
0  993.0                 4.761905         49.65     6.6  


In [36]:
# Connect to an on-disk DuckDB instance (replace 'your_database.db' with your desired database file path)
con = duckdb.connect(database='first_database.db')

In [37]:
con

In [38]:
con.execute(f"""
CREATE TABLE IF NOT EXISTS sales AS
SELECT *
FROM read_csv_auto('{csv_file_path}')
""")

In [39]:
# Verify the table contents by querying the first few rows
df = con.execute("SELECT * FROM sales LIMIT 5").fetchdf()

In [40]:
df

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [48]:
con.execute("""
SELECT "City", "Customer type", "Total"
FROM sales
ORDER BY "Total" DESC
LIMIT 1
""").fetchdf()

,City,Customer type,Total
0,Naypyitaw,Member,1042.65


In [49]:
con.execute("""
CREATE TABLE customer_info AS
SELECT "Branch", "City", "Customer type", "Gender", "Product line"
FROM sales
""")

In [50]:
con.execute("select * from customer_info").fetchdf()

,Branch,City,Customer type,Gender,Product line
0,A,Yangon,Member,Female,Health and beauty
1,C,Naypyitaw,Normal,Female,Electronic accessories
2,A,Yangon,Normal,Male,Home and lifestyle
3,A,Yangon,Member,Male,Health and beauty
4,A,Yangon,Normal,Male,Sports and travel
...,...,...,...,...,...
995,C,Naypyitaw,Normal,Male,Health and beauty
996,B,Mandalay,Normal,Female,Home and lifestyle
997,A,Yangon,Member,Male,Food and beverages
998,A,Yangon,Normal,Male,Home and lifestyle


In [51]:
# Create views for the separate data subsets
con.execute("""
CREATE VIEW info_view AS
SELECT "City", "Customer type", "Gender"
FROM customer_info
""")

In [52]:
con.execute("""
CREATE VIEW general_view AS
SELECT "Branch", "Product line"
FROM customer_info
""")

In [53]:
# Fetch the data from the views
info_df = con.execute("SELECT * FROM info_view").fetchdf()
general_df = con.execute("SELECT * FROM general_view").fetchdf()


In [56]:
# Convert the dataframes to dictionaries
info_dict = info_df.to_dict(orient='list')
general_dict = general_df.to_dict(orient='list')

In [57]:
# Create the final dictionary with the required structure
final_dict = {
    "info": info_dict,
    "general": general_dict
}

In [58]:
# Print the final dictionary to verify
print(final_dict)

{'info': {'City': ['Yangon', 'Naypyitaw', 'Yangon', 'Yangon', 'Yangon', 'Naypyitaw', 'Yangon', 'Naypyitaw', 'Yangon', 'Mandalay', 'Mandalay', 'Mandalay', 'Yangon', 'Yangon', 'Yangon', 'Mandalay', 'Yangon', 'Yangon', 'Yangon', 'Mandalay', 'Naypyitaw', 'Mandalay', 'Mandalay', 'Yangon', 'Yangon', 'Yangon', 'Mandalay', 'Yangon', 'Mandalay', 'Yangon', 'Mandalay', 'Mandalay', 'Mandalay', 'Yangon', 'Naypyitaw', 'Naypyitaw', 'Yangon', 'Yangon', 'Naypyitaw', 'Mandalay', 'Mandalay', 'Naypyitaw', 'Mandalay', 'Naypyitaw', 'Naypyitaw', 'Mandalay', 'Mandalay', 'Mandalay', 'Mandalay', 'Naypyitaw', 'Naypyitaw', 'Yangon', 'Mandalay', 'Naypyitaw', 'Mandalay', 'Naypyitaw', 'Yangon', 'Yangon', 'Yangon', 'Naypyitaw', 'Naypyitaw', 'Naypyitaw', 'Mandalay', 'Yangon', 'Mandalay', 'Yangon', 'Naypyitaw', 'Mandalay', 'Yangon', 'Yangon', 'Naypyitaw', 'Naypyitaw', 'Mandalay', 'Naypyitaw', 'Yangon', 'Naypyitaw', 'Naypyitaw', 'Yangon', 'Naypyitaw', 'Naypyitaw', 'Naypyitaw', 'Mandalay', 'Naypyitaw', 'Naypyitaw', 'Nayp